# **PRÀCTICA 1: Identificació d'idioma**

In [1]:
import math
import nltk
from nltk.collocations import TrigramCollocationFinder
import json

In [3]:
# Load trigrams and unique characters from json files
def dict_string_to_tuple(d: dict) -> dict:
    return {key: {tuple(k): v for k, v in value.items()} for key, value in d.items()}

trigrams = dict_string_to_tuple(json.load(open('./weights/trigrams.json')))
unique_chars = json.load(open('./weights/unique_chars.json'))

In [4]:
# Functions for Lidstone smoothing
def lidstone_smooth(language: str, trigram: tuple, lambda_value: float = 0.5):
    counts = trigrams[language]
    vocab = unique_chars[language] ** 3
    total = sum(counts.values())
    try:
        number = counts[trigram]
    except:
        number = 0
    probs = (number + lambda_value) / (total + lambda_value * vocab)
    return probs

def lidstone_total(text: str, language: str):
    trigram_finder = TrigramCollocationFinder.from_words(text)
    prob_sec = 0
    for trigram, num_instances in trigram_finder.ngram_fd.items():
        prob_sec += num_instances * math.log(lidstone_smooth(trigram=trigram, language=language))
    return prob_sec

In [7]:
# Test the Lidstone smoothing
text = 'Example'
probs = []
languages = {"deu_trn.txt": "Deutsch (German)", 
             "eng_trn.txt": "English (English)", 
             "fra_trn.txt": "Français (French)", 
             "ita_trn.txt": "Italiano (Italian)", 
             "spa_trn.txt": "Español (Spanish)", 
             "nld_trn.txt": "Nederlands (Dutch)"}
for language in trigrams.keys():
    prob = lidstone_total(text, language)
    probs.append((language, prob))
max_probs = max(probs, key = lambda x: x[1])
print(max_probs)
probs = sorted(probs, key = lambda x: x[1], reverse=True)

('eng_trn.txt', -49.787416303726516)


In [8]:
# Normalize the probabilities and print the results
exp = [math.exp(p[1] - max_probs[1]) for p in probs]
normalize = [(e / sum (exp))*100 for e in exp]
lang = [languages[p[0]] for p in probs]
for e in range(len(lang)):
    print(f"{lang[e]}: {normalize[e]}")

English (English): 79.88980661894925
Français (French): 13.050489362263882
Español (Spanish): 6.337352147331384
Nederlands (Dutch): 0.6762405300571243
Italiano (Italian): 0.041759620819945685
Deutsch (German): 0.004351720578416145


In [9]:
import nltk
from nltk.util import ngrams
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import KneserNeyInterpolated

# Asegúrate de haber descargado los paquetes necesarios de NLTK la primera vez
# nltk.download('punkt')

# Ejemplo de texto para entrenar el modelo
text = "Este es un ejemplo de texto para demostrar cómo podemos aplicar suavizado de Kneser-Ney a trigramas. Este método ayuda a manejar n-gramas no observados."

# Tokenización del texto
tokens = nltk.word_tokenize(text, language='spanish')

# Preparación de los datos para el modelo de trigramas
train_data, padded_sents = padded_everygram_pipeline(3, [tokens])

# Creación del modelo de trigramas con Kneser-Ney Smoothing
model = KneserNeyInterpolated(3)  # El argumento es el orden del n-grama, 3 para trigramas

# Entrenamiento del modelo con los datos preparados
model.fit(train_data, padded_sents)

# Ejemplo de uso del modelo para generar la siguiente palabra después de un contexto dado
context = ['este', 'es']
print("Palabra(s) siguiente(s) más probable(s) después de 'este es':", model.generate(text_seed=context))

# Probabilidad de un trigram específico
trigram = ['para', 'demostrar', 'cómo']
print("Probabilidad del trigram 'para demostrar cómo':", model.score(trigram[-1], trigram[:-1]))


Palabra(s) siguiente(s) más probable(s) después de 'este es': un
Probabilidad del trigram 'para demostrar cómo': 0.9903448275862069
